<a href="https://colab.research.google.com/github/immsswd/data_science/blob/main/Data_Science_in_Retail_Product_Packaging_(MBA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%load_ext rpy2.ipython

Untuk memulai kode dengan **R** harus didahului dengan %%R, untuk menginisiasi bahwa kode akan menggunakan bahasa R

In [3]:
%%R
# instal library association rules/arules
install.packages("arules")

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/arules_1.6-7.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 1824587 bytes (1.7 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[writ

In [ ]:
%%R
library(arules)

In [5]:
%%R
transaksi_tabular <- read.transactions(file="https://storage.googleapis.com/dqlab-dataset/transaksi_dqlab_retail.tsv", format="single", sep="\t", cols=c(1,2), skip=1)
transaksi_tabular

transactions in sparse format with
 3450 transactions (rows) and
 69 items (columns)


In [ ]:
%%R
write(transaksi_tabular, file="test_project_retail_1.txt", sep = ",")

**Objektif 1**: Statistik ***Top 10*** Product:

In [ ]:
%%R
# menampilkan itemFrequency dari data
data_top <- itemFrequency(transaksi_tabular, type="absolute")

# sorting data per product dari besar ke kecil
data_top <- sort(data_top, decreasing=TRUE)

# Slicing hanya Top 10 saja
data_top <- data_top[1:10]

print(data_top)
#names(data_top)

             Shampo Biasa             Serum Vitamin         Baju Batik Wanita 
                     2075                      1685                      1312 
        Baju Kemeja Putih      Celana Jogger Casual               Cover Koper 
                     1255                      1136                      1086 
       Sepatu Sandal Anak Tali Pinggang Gesper Pria       Sepatu Sport merk Z 
                     1062                      1003                       888 
             Wedges Hitam 
                      849 


Mengubah format data menjadi frame menggunakan function 

```
data.frame()
```
Agar lebih mudah dibaca untuk mendapatkan insight dari data


In [ ]:
%%R
# mengubah ke dataframe
data_top <- data.frame("Nama Produk"=names(data_top), "Jumlah"=data_top, row.names = NULL)
print(data_top)

                 Nama.Produk Jumlah
1               Shampo Biasa   2075
2              Serum Vitamin   1685
3          Baju Batik Wanita   1312
4          Baju Kemeja Putih   1255
5       Celana Jogger Casual   1136
6                Cover Koper   1086
7         Sepatu Sandal Anak   1062
8  Tali Pinggang Gesper Pria   1003
9        Sepatu Sport merk Z    888
10              Wedges Hitam    849


**Objektif 2**: Statistik ***Bottom10*** Produk (yaitu produk-produk yang paling kurang populuer dari bawah), Perbedaan dari ***Top 10*** adalah:


```
# sorting data per product dari besar ke kecil
data_top <- sort(data_top, decreasing=FALSE)
```
`decreasing` nya = `FALSE`


In [ ]:
%%R
# menampilkan itemFrequency dari data
data_bottom <- itemFrequency(transaksi_tabular, type="absolute")

# sorting data per product dari besar ke kecil
data_bottom <- sort(data_bottom, decreasing=FALSE)

# Slicing hanya Top 10 saja
data_bottom <- data_bottom[1:10]

print(data_bottom)
#names(data_bottom)

   Celana Jeans Sobek Pria               Tas Kosmetik 
                         9                         11 
              Stripe Pants                   Pelembab 
                        19                         24 
    Tali Ban Ikat Pinggang Baju Renang Pria Anak-anak 
                        27                         32 
                  Hair Dye         Atasan Baju Belang 
                        46                         56 
Tas Sekolah Anak Perempuan              Dompet Unisex 
                        71                         75 


In [ ]:
# Agar lebih enak dibaca lagi, kita ubah data ke format frame
%%R
# mengubah ke dataframe
data_bottom <- data.frame("Nama Produk"=names(data_bottom), "Jumlah"=data_bottom, row.names = NULL)
print(data_bottom)

                  Nama.Produk Jumlah
1     Celana Jeans Sobek Pria      9
2                Tas Kosmetik     11
3                Stripe Pants     19
4                    Pelembab     24
5      Tali Ban Ikat Pinggang     27
6  Baju Renang Pria Anak-anak     32
7                    Hair Dye     46
8          Atasan Baju Belang     56
9  Tas Sekolah Anak Perempuan     71
10              Dompet Unisex     75


Berikut adalah Permintaan kombinasi produk yang menarik dari Client:

*   Memiliki asosiasi atau hubungan erat.
*   Kombinasi produk minimal 2 item, dan maksimum 3 item.
*   Kombinasi produk itu muncul setidaknya 10 dari dari seluruh transaksi.
*   Memiliki tingkat confidence minimal 50 persen.

In [6]:
%%R
combination <- apriori(transaksi_tabular, parameter = list(supp=10/length(transaksi_tabular), minlen=2, maxlen=3, conf=0.5))
print(combination)

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime     support minlen
        0.5    0.1    1 none FALSE            TRUE       5 0.002898551      2
 maxlen target  ext
      3  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 10 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[69 item(s), 3450 transaction(s)] done [0.01s].
sorting and recoding items ... [68 item(s)] done [0.00s].
creating transaction tree ... done [0.00s].
checking subsets of size 1 2 3 done [0.01s].
writing ... [4637 rule(s)] done [0.01s].
creating S4 object  ... done [0.00s].
set of 4637 rules 


Terdapat `4637` association rules

In [7]:
%%R
combination_result <- head(combination, n = 10, by = "lift")
combination_result
inspect(combination_result)

     lhs                             rhs                              support confidence    coverage     lift count
[1]  {Tas Makeup,                                                                                                  
      Tas Pinggang Wanita}        => {Baju Renang Anak Perempuan} 0.010434783  0.8780488 0.011884058 24.42958    36
[2]  {Tas Makeup,                                                                                                  
      Tas Travel}                 => {Baju Renang Anak Perempuan} 0.010144928  0.8139535 0.012463768 22.64629    35
[3]  {Tas Makeup,                                                                                                  
      Tas Ransel Mini}            => {Baju Renang Anak Perempuan} 0.011304348  0.7358491 0.015362319 20.47322    39
[4]  {Sunblock Cream,                                                                                              
      Tas Pinggang Wanita}        => {Kuas Makeup }               0.0162

# Mencari paket produk yang bisa dipasangkan dengan **item** Slow-Moving (Pergerakan Penjualannya lambat)
Kadang kala item ini belum tentu tidak laku, hanya saja mungkin harganya tidak bagus dan jarang dibutuhkan jika dijual satuan. Nah, jika tidak dijual satuan si *data scientist* perlu mencari asosiasi yang kuat dari item produk ini dengan produk lain, jika dipaketkan akan menjadi kombinasi yang lebih menarik.

**Objektif: Mencari kombinasi yang dapat dipaketkan dengan produk "Tas Maekup" dan "Baju Renang Pria Anak-anak" karena kedua produk tersebut stoknya masih banyak.**